In [5]:
import numpy as np
import gym
from gym import Env, spaces, utils
import pygame
from typing import Optional
from io import StringIO
from contextlib import closing
from gym.spaces import Discrete
from gym.wrappers import FlattenObservation
import os

In [6]:
from env_2048 import *

In [3]:
env = BlockDoubleEnv()
env.reset()
env._boardstate = [[0.0, 2.0, 2.0, 2.0],
                    [0.0, 0.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0, 0.0]]
new_state, reward, done, info = env.step(0)
print(env.render())

[2. 2. 2.]
[2. 4.]
[0. 0. 2. 4.]
[]
[0. 0. 0. 0.]
[]
[0. 0. 0. 0.]
[]
[0. 0. 0. 0.]
[0.0, 0.0, 2.0, 4.0]
[2.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]



In [10]:
from IPython.display import clear_output
env = BlockDoubleEnv()
env.reset()  # reset enviornment to default state
score = 0
while True:
    check = 0
    print(env.render())
    key = input("w,a,s,d\n").lower()
    if key == "w":
        action = 3
    elif key == "d":
        action = 0
    elif key == "s":
        action = 1
    elif key == "a":
        action = 2
    else:
        break
    clear_output()
    new_state, reward, done, info = env.step(action)



[0.0, 0.0, 0.0, 4.0]
[2.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]



In [37]:
turns = 2
for turn in range(1,turns+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode:{turn}, Score:{reward}")


KeyboardInterrupt: 

In [6]:
score = 0
print(env.render())
new_state, reward, done, info = env.step(0)
score += reward
print(env.render())
new_state, reward, done, info = env.step(1)
score += reward
print(env.render())
new_state, reward, done, info = env.step(2)
score += reward
print(env.render())
new_state, reward, done, info = env.step(3)
score += reward
print(env.render())
new_state, reward, done, info = env.step(0)
score += reward
print(env.render())
new_state, reward, done, info = env.step(1)
score += reward


[2.0, 0.0, 2.0, 0.0]
[0.0, 0.0, 2.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]

1
[2.0, 2.0, 2.0, 0.0]
[0.0, 0.0, 4.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]

2
1
[0.0, 4.0, 4.0, 2.0]
[0.0, 0.0, 2.0, 4.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]

1
1
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[2.0, 0.0, 4.0, 2.0]
[0.0, 4.0, 2.0, 4.0]

2
1
2
1
[4.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[2.0, 4.0, 2.0, 0.0]
[4.0, 2.0, 4.0, 0.0]

1
1
2
1
[4.0, 4.0, 2.0, 0.0]
[2.0, 2.0, 4.0, 0.0]
[4.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]

1
2
1
2
1


In [16]:
from env_2048 import *

env = BlockDoubleEnv()
#2048 = 2^11
number_of_powers = 11
STATES = env.size * env.size * (number_of_powers + 1)
ACTIONS = env.action_space.n
Q = np.zeros((STATES, ACTIONS))  # create a matrix with all 0 values 

EPISODES = 2000 # how many times to run the enviornment from the beginning
MAX_STEPS = 100  # max number of steps allowed for each run of enviornment

LEARNING_RATE = 0.81  # learning rate
GAMMA = 0.96

RENDER = True # if you want to see training set to true

epsilon = 0.9

In [19]:
STATES

192

In [18]:
env.reset()

array([[0., 0., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [17]:
rewards = []
for episode in range(EPISODES):

  state = env.reset()
  for _ in range(MAX_STEPS):
    
    if RENDER:
      env.render()

    if np.random.uniform(0, 1) < epsilon:
      action = env.action_space.sample()  
    else:
      action = np.argmax(Q[state, :])

    next_state, reward, done, _ = env.step(action)

    Q[state, action] = Q[state, action] + LEARNING_RATE * (reward + GAMMA * np.max(Q[next_state, :]) - Q[state, action])

    state = next_state

    if done: 
      rewards.append(reward)
      epsilon -= 0.001
      break  # reached goal

print(Q)
print(f"Average reward: {sum(rewards)/len(rewards)}:")
# and now we can see our Q values!

IndexError: arrays used as indices must be of integer (or boolean) type